* Grab TRD loci of all crosses
* do the SNPRelate phylogeny for each locus

In [1]:
library(data.table)
library(tidytable)
library(ggplot2)


Attaching package: 'tidytable'


The following objects are masked from 'package:data.table':

    between, first, fread, last


The following objects are masked from 'package:stats':

    dt, filter, lag


The following object is masked from 'package:base':

    %in%




In [2]:
library(SNPRelate)
library(ape)

# Get paths of input files
my.vcf <- "/home/jnrunge/data/trd/full2489Matrix.vcf.gz"
subset_samples = "none"

if(subset_samples == "none"){
    subset_samples=NULL
    prefix <- basename(my.vcf)
    }else{
    prefix <- basename(subset_samples)
    subset_samples=readLines(subset_samples)
    }

setwd(dirname(my.vcf))
snpgdsVCF2GDS(my.vcf, paste0(prefix, ".gds"), ignore.chr.prefix="chromosome")


Loading required package: gdsfmt

SNPRelate -- supported by Streaming SIMD Extensions 2 (SSE2)


Attaching package: 'ape'


The following object is masked from 'package:tidytable':

    where




Start file conversion from VCF to SNP GDS ...
Method: extracting biallelic SNPs
Number of samples: 2489
Parsing "/home/jnrunge/data/trd/full2489Matrix.vcf.gz" ...
	import 2216200 variants.
+ genotype   { Bit2 2489x2216200, 1.3G } *
Optimize the access efficiency ...
Clean up the fragments of GDS file:
    open the file 'full2489Matrix.vcf.gz.gds' (1.3G)
    # of fragments: 275
    save to 'full2489Matrix.vcf.gz.gds.tmp'
    rename 'full2489Matrix.vcf.gz.gds.tmp' (1.3G, reduced: 3.0K)
    # of fragments: 20


In [3]:
genofile <- snpgdsOpen(paste0(prefix, ".gds"))


In [8]:
if(!file.exists("pca_global.rds")){
    pca_global <- snpgdsPCA(genofile, num.thread=1)
    saveRDS(pca_global, file = "pca_global.rds")
}else{
    pca_global=readRDS(file = "pca_global.rds")
}

In [9]:
pca_global.percent <- pca_global$varprop*100
head(round(pca_global.percent, 2))

[1] 4.92 2.52 1.77 1.54 1.50 1.46

In [10]:
tab_global <- data.frame(sample.id = pca_global$sample.id,
    EV1 = pca_global$eigenvect[,1],    # the first eigenvector
    EV2 = pca_global$eigenvect[,2],    # the second eigenvector
    stringsAsFactors = FALSE)

In [ ]:
crosses=readLines("~/data/trd/mapped_reads/TRD.vcf.gz.samples")
crosses=crosses[startsWith(crosses, "YJNRC") | startsWith(crosses, "Chris")]
crosses

In [ ]:
for(c in crosses){
    if(!file.exists(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-TRD_regions.csv.gz"))){
        next
    }
    p_global<-ggplot(tab_global, aes(EV2,EV1, color=sample.id=="BAH",size=sample.id=="BAH"))+
geom_point(shape=1)+theme_bw(18)+scale_color_manual(values=c("TRUE"="red","FALSE"="blue"))
    print(p_global)
    
    AS=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz.allelesharing.csv.gz"))

    TRD=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz"))

    TRD_loci=fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-TRD_regions.csv.gz"))

    for(i in 1:nrow(TRD_loci)){
        if(TRD_loci$chr_start[i]!=TRD_loci$chr_end[i]){
            stop("chr overlapping TRD")
        }
        TRD_subset=filter(TRD, chr== TRD_loci$chr_start[i] & global_pos >= TRD_loci$global_start[i] & global_pos <= TRD_loci$global_end[i])
        df_AS_filtered=filter(AS, `#CHROM` == TRD_loci$chr_start[i], POS %in% TRD_subset$pos)

        snpset=which(read.gdsn(index.gdsn(genofile, "snp.position"))%in%TRD_subset$pos & 
                     paste0("chromosome",read.gdsn(index.gdsn(genofile, "snp.chromosome"))) == TRD_loci$chr_start[i])

        pca_TRD <- snpgdsPCA(genofile, num.thread=1, snp.id = snpset)

        pca_TRD.percent <- pca_TRD$varprop*100

        tab <- data.frame(sample.id = pca_TRD$sample.id,
            EV1 = pca_TRD$eigenvect[,1],    # the first eigenvector
            EV2 = pca_TRD$eigenvect[,2],    # the second eigenvector
            stringsAsFactors = FALSE)

        p<-ggplot(tab, aes(EV2,EV1, color=sample.id=="BAH",size=sample.id=="BAH"))+
        geom_point(shape=1)+theme_bw(18)+scale_color_manual(values=c("TRUE"="red","FALSE"="blue"))
        print(p)

    }

}